## matplotlib 실행시 jupyter kernel died 문제
- 에러 내용
```
Initializing libiomp5md.dll, but found libiomp5 already initialized.

Hint This means that multiple copies of the OpenMP runtime have been linked into the program. That is dangerous, since it can degrade performance or cause incorrect results. The best thing to do is to ensure that only a single OpenMP runtime is linked into the process, e.g. by avoiding static linking of the OpenMP runtime in any library.

As an unsafe, unsupported, undocumented workaround you can set the environment variable KMP_DUPLICATE_LIB_OK=TRUE to allow the program to continue to execute, but that may cause crashes or silently produce incorrect results. For more information, please see http://www.intel.com/software/products/support/.

libiomp5md.dll을 초기화하고 있지만 libiomp5가 이미 초기화되어 있습니다.

힌트 이것은 OpenMP 런타임의 여러 복사본이 프로그램에 연결되었다는 것을 의미합니다. 성능을 저하시키거나 잘못된 결과를 초래할 수 있기 때문에 위험합니다. 최선의 방법은 예를 들어 라이브러리에서 OpenMP 런타임의 정적 연결을 피함으로써 단일 OpenMP 런타임만 프로세스에 연결되도록 하는 것입니다.

안전하지 않고 지원되지 않으며 문서화되지 않은 해결 방법으로 환경 변수 KMP_DUPLICATE_LIB_OK=TRUE를 설정하여 프로그램을 계속 실행할 수 있지만 충돌을 유발하거나 자동으로 잘못된 결과를 생성할 수 있습니다. 자세한 내용은 http://www.intel.com/software/products/support/ 를 참조하십시오.
```
- 텐서플로우 사용시 matplotlib.pyplot을 사용하면 둘 중 하나는 kernel died 현상이 발생함
- plt로 이미지를 나타낸 후 tensorflow을 실행하거나, tensorflow을 실행 후 plt 이미지를 실행하려고 하면 kernel이 죽었다가 다시 시작되는 문제가 발생

### 문제 원인
- 여러 라이브러리들이 각각의 OpenMP를 로드하려고 할 떄 충돌이 일어나고, 여기에서 intel의 math kernel library(mkl)을 중복으로 사용하면서 문제가 발생하는 것 같다. 

### 문제 해결
- 1번을 사용해서 우선적으로 해결할 수 있었다.
- 같은 에러를 겪은 다른 사람들은 1번 방법으로 해결이 안되는 경우가 있는 것 같다.
- 1. 환경변수의 값을 직접 설정 
   - `os.environ["KMP_DUPLICATE_LIB_OK"] = "True"`
   - 환경변수 중 KMP_DUPlICATE_LIB_OK 는 intel의 math kernel library(mkl)을 사용할 때 여러 OpenMP 라이브러리가 중복 로드 될 때 이것을 허용하도록 한다.
   - 그러나 이것은 완전한 해결 방법이 아니라고 함. 임시적인 해결. 올바른 OpenMP 라이브러리를 사용하도록 환경을 설정해야한다.
- 2. mkl을 중복 설치하지 못하도록 방지
   - `conda install nomkl`
   - anaconda에서 지원하지 않는 패키지 같음, 설치 안됨
   - mkl을 중복해서 설치하지 못하도록 막아주는 패키지
   - 이것도 완전한 방법은 아니라고 함...
- 3. 환경변수 설정에서 변수를 직접 변경
   - 사용자변수와 시스템변수에서 각각 KMP_DUPLICATE_LIB_OK, TRUE를 설정
   
### 기타
- 운영체제의 환경변수를 설정하기 위해서 os 패키지를 사용한다. 환경변수를 다루는 방법에 대한 참고 사이트
   - https://www.daleseo.com/python-os-environ/

### os를 사용하여 환경변수 설정

In [1]:
import os

In [3]:
os.getenv("HOME")

'C:\\Users\\saint hong'

In [5]:
os.getenv("SHELL")

'C:\\Program Files\\Git\\usr\\bin\\bash.exe'

In [7]:
os.environ["KMP_DUPLICATE_LIB_OK"] = "True"

In [9]:
os.getenv("KMP_DUPLICATE_LIB_OK")

'True'